<a href="https://colab.research.google.com/github/shaktsin/coolml/blob/master/attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00


In [3]:
import tiktoken

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [8]:
from huggingface_hub import login

login(os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [9]:
import datasets
miracl = datasets.load_dataset('miracl/miracl', 'hi')

README.md:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

miracl.py:   0%|          | 0.00/6.49k [00:00<?, ?B/s]

The repository for miracl/miracl contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/miracl/miracl.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)-hi/topics/topics.miracl-v1.0-hi-dev.tsv:   0%|          | 0.00/52.0k [00:00<?, ?B/s]

(…).0-hi/qrels/qrels.miracl-v1.0-hi-dev.tsv:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

...

docs-0.jsonl.gz:   0%|          | 0.00/95.7M [00:00<?, ?B/s]

docs-1.jsonl.gz:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating testB split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
len(miracl['train'])
miracl['train'][0]['negative_passages']


[{'docid': '1698#0',
  'text': "भारत के उत्तर-पूर्व में सात राज्य हैं। इन्हें 'सात-बहनें' या 'सेवन-सिस्टर्स' के नाम से भी जाना जाता है। इन राज्यों में 255,511 वर्ग किलोमीटर (98,653 वर्ग मील), या भारत के कुल क्षेत्रफल का लगभग सात प्रतिशत के एक क्षेत्र को कवर किया हुआ है। वर्ष 2011 में 44,98 लाख की आबादी थी, जो कि भारत के कुल आबादी की \xa03.7 प्रतिशत थी। हालांकि वहाँ सात राज्यों के भीतर महान जातीय और धार्मिक विविधता है, लेकिन राजनीतिक, सामाजिक और आर्थिक क्षेत्रों में समानता भी है।",
  'title': 'उत्तर-पूर्वी राज्य'}]

In [11]:
with open('corpus.text', 'w') as f:
  for i in range(len(miracl['train'])):
    f.write(miracl['train']['query'][i]+ '\n')
    for j in range(len(miracl['train']['positive_passages'][i])):
      doc = miracl['train']['positive_passages'][i][j]
      f.write(doc['title'] + '\n')
      f.write(doc['text'] + '\n')
    for j in range(len(miracl['train']['negative_passages'][i])):
      doc = miracl['train']['negative_passages'][i][j]
      f.write(doc['title'] + '\n')
      f.write(doc['text'] + '\n')


In [14]:
!ls

corpus.text  sample_data


In [15]:
!head corpus.text

भारत में कुल कितने राज्य है?
भारतीय राज्य पशुओं की सूची
भारत, आधिकारिक भारत गणराज्य एक दक्षिण एशियाई देश है। यह 28 राज्यों और 8 केन्द्र शासित प्रदेशों से मिलकर बना है। सभी भारतीय अपनी स्वयं की सरकार रखते हैं और केन्द्रशासित प्रदेश केन्द्र सरकार के अधिकार क्षेत्र में आते हैं। अधिकतर अन्य देशों की तरह भारत में भी राष्ट्रीय प्रतीक पाये जाते हैं। राष्ट्रीय प्रतीकों के अतिरिक्त सभी भारतीय राज्य और केन्द्र शासित प्रदेश अपनेखुद की राज्य मोहर और प्रतीक रखते हैं जिसमें राज्य पशु, पक्षी, पेड़, फूल आदि शामिल हैं। भारत के सभी राज्य पशुओं की सूची निचे दी गयी है।  ! width=98px|केन्द्रशासित प्रदेश ! width=138px|सामान्य नाम ! width=155px|वैज्ञानिक नाम ! width=155px|चित्र
भारत के राज्य तथा केन्द्र-शासित प्रदेश
भारत राज्यों का एक संघ है। इसमें 28 राज्य और 8 केन्द्र शासित प्रदेश हैं। ये राज्य और केन्द्र शासित प्रदेश पुनः जिलों और अन्य क्षेत्रों में बाँटे गये हैं।
भारतीय राज्य वृक्षों की सूची
भारत आधिकारिक तौर पर भारत गणराज्य दक्षिण एशिया का एक देश है। यह 28 राज्यों और 8 केंद्र शासित प्रदेशों से बना है। स

In [ ]:
enc = tiktoken.get_encoding("gpt2")
ids = enc.encode("भारतीय राज्य पशुओं की सूची")
enc.decode(ids)

'भारतीय राज्य पशुओं की सूची'

In [ ]:
enc.n_vocab

50257

In [13]:
text = 'Hello World'
tokens = text.encode('utf-8')
tokens = list(map(int, tokens))
print(tokens)


[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100]


In [ ]:
len(ids)

42

In [ ]:
# token

In [ ]:
n_embd = 64
block_size = 32
dropout = 0.0

class Head(nn.Module):

  def __init__(self, head_size=16):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def __forward__(self, x):
    B,T,C = x.shape
    k = self.key(self.key)
    q = self.query(self.query)
